# 세종시 주택시장 분석
## 팀명: 세종대왕

- - -

2012년 새롭게 출범한 세종특별자치시는, **'행정중심복합도시'** 라는 우리나라에서 처음으로 시도하는 특별한 형태의 도시입니다.   
또한 작년, '세종천도설' 등 여러 이슈에 휩싸이며 주택의 거래량이 가장 빠르게 증가하던 곳이기도 합니다.   
그래서 저희는 주어진 데이터를 통해 **세종시의 주택시장이 어떤 특징을 가지고 있는지**를 중심으로 명료한 시각화 모델을 통해 제시하고자 합니다. 

- - -

## 코드 목차

### 0. Import
  
### 1. Data Preprocessing  
  
- 내부 데이터 전처리    
- 외부 데이터 전처리    
  
### 2. Visualization  
  
- 시기 별 이슈 확인 (WordCloud)  
- 주택 및 거래 관련  
- 인구 관련
  
### 3. Modeling   

- 지수 생성  
- LDA  
- Multiple Regression Analysis  
- Oaxaca Decomposition  
- - -

# 0. Import

In [ ]:
from geoband.API import *
import folium
import json
import geopandas as gpd
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import pydeck as pdk
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import warnings
warnings.filterwarnings("ignore")
col = px.colors.qualitative.Pastel

from geopandas import GeoDataFrame
from pandas import DataFrame
import deckgljupyter.Layer as deckgl

from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import random
from sklearn.preprocessing import MinMaxScaler
np.random.seed(seed=42)
from sklearn.decomposition import PCA

random.seed(42)
import hdbscan

In [ ]:
GetCompasData('SBJ_2102_001', '2', '2.세종시_표제부.csv')
GetCompasData('SBJ_2102_001', '3', '3.세종시_아파트(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '4', '4.세종시_연립다세대(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '5', '5.세종시_단독다가구(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '6', '6.세종시_오피스텔(매매)_실거래가.csv')
GetCompasData('SBJ_2102_001', '7', '7.세종시_아파트(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '8', '8.세종시_연립다세대(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '9', '9.세종시_단독다가구(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '10', '10.세종시_오피스텔(전월세)_실거래가.csv')
GetCompasData('SBJ_2102_001', '14', '14.세종시_상권정보.csv')
GetCompasData('SBJ_2102_001', '18', '18.세종시_개별공시지가(2017~2020).csv')
GetCompasData('SBJ_2102_001', '19', '19.세종시_연령별_거주인구정보_격자.geojson')
GetCompasData('SBJ_2102_001', '20', '20.세종시_전입자수.csv')
GetCompasData('SBJ_2102_001', '21', '21.세종시_전출자수.csv')
GetCompasData('SBJ_2102_001', '22', '22.세종시_연령별_인구현황.csv')
GetCompasData('SBJ_2102_001', '28', '28.세종시_지역별_세대원수별_세대수.csv')
GetCompasData('SBJ_2102_001', '29', '29.세종시_거주의사(향후).csv')
GetCompasData('SBJ_2102_001', '31', '31.세종시_법정경계(읍면동).geojson')
GetCompasData('SBJ_2102_001', '32', '32.세종시_행정경계(읍면동).geojson')

In [ ]:
data2 = pd.read_csv('2.세종시_표제부.csv',encoding='utf-8')
data3 = pd.read_csv('3.세종시_아파트(매매)_실거래가.csv', encoding='utf-8')
data4 = pd.read_csv('4.세종시_연립다세대(매매)_실거래가.csv',  encoding='utf-8')
data5 = pd.read_csv('5.세종시_단독다가구(매매)_실거래가.csv', encoding='utf-8')
data6 = pd.read_csv('6.세종시_오피스텔(매매)_실거래가.csv',  encoding='utf-8')
data7 = pd.read_csv('7.세종시_아파트(전월세)_실거래가.csv',  encoding='utf-8')
data8 = pd.read_csv('8.세종시_연립다세대(전월세)_실거래가.csv', encoding='utf-8')
data9 = pd.read_csv('9.세종시_단독다가구(전월세)_실거래가.csv', encoding='utf-8')
data10 = pd.read_csv('10.세종시_오피스텔(전월세)_실거래가.csv', encoding='utf-8')
data14 = pd.read_csv('14.세종시_상권정보.csv', encoding= 'utf-8')
data18 = pd.read_csv('18.세종시_개별공시지가(2017~2020).csv', encoding='utf-8')
data19 = gpd.read_file('19.세종시_연령별_거주인구정보_격자.geojson')
data20 = pd.read_csv('20.세종시_전입자수.csv', encoding='utf-8')
data21 = pd.read_csv('21.세종시_전출자수.csv', encoding='utf-8')
data22 = pd.read_csv('22.세종시_연령별_인구현황.csv', encoding='utf-8')
data28 = pd.read_csv('28.세종시_지역별_세대원수별_세대수.csv', encoding='utf-8')
data29 = pd.read_csv('29.세종시_거주의사(향후).csv', encoding='utf-8')
data31 = gpd.read_file('31.세종시_법정경계(읍면동).geojson')
data32 = gpd.read_file('32.세종시_행정경계(읍면동).geojson')
geo = gpd.read_file('final_geo.geojson')

- - -
# 1. Data Preprocessing

## 1) 매매, 전월세, 공시지가 데이터 전처리

- 지번주소가 다른 방식(ex. 본번, 부번)으로 제공되어있는 

  매매(3,4,5,6), 전월세(7,8,9,10), 공시지가(18) raw 데이터를 동일한 지번주소로 통합.

In [ ]:
# 시군구, 본번, 부번 합쳐서 지번주소 만드는 함수 정의

def get_address(data):
    data = data.reset_index()
    df = data
    df['부번']=df['부번'].astype(str)
    df['본번']=df['본번'].astype(str)
    df['시군구']=df['시군구'].astype(str)
    
    drop = df.loc[(df['본번'] == '0') & (df['부번'] == '0')].index
    df = df.drop(drop)
    
    # 유형 별로 주소 데이터 나누기
    df1 = df[df['본번'] == '0'][['index','시군구','본번','부번']]
    df2 = df[df['부번'] == '0'][['index','시군구','본번','부번']]
    df3 = df.loc[(df['본번'] != '0') & (df['부번'] != '0')][['index','시군구','본번','부번']]
    
    # 각 유형에 따라 번지 부여
    df1['입력주소'] = " " + df1['시군구'] + " " + df1['부번'] + "번지"
    df2['입력주소'] = " " + df2['시군구'] + " " + df2['본번'] + "번지"
    df3['입력주소'] = " " + df3['시군구']+ " " + df3['본번'] + "-" + df3['부번'] + "번지"
    
    # 3개의 데이터 Merging
    from functools import reduce
    dfs = [df1,df2,df3]
    df_merge = reduce(lambda left, right: 
                      pd.merge(left, right, on=['index','입력주소'], how='outer'), dfs)
    df_merge = df_merge[['index','입력주소']]
    df_final = pd.merge(data,df_merge,on = 'index',how='outer')
    return df_final

### 매매, 전월세 데이터 지번주소 뽑아내기

In [ ]:
# 각 데이터에서 지번 주소들 뽑아내기

df1 = get_address(data3)[['index','입력주소']]
df2 = get_address(data4)[['index','입력주소']]
df3 = get_address(data6)[['index','입력주소']]
df4 = get_address(data7)[['index','입력주소']]
df5 = get_address(data8)[['index','입력주소']]
df6 = get_address(data10)[['index','입력주소']]

# Data Merging
from functools import reduce
dfs = [df1,df2,df3,df4,df5,df6]
df_merge = reduce(lambda left, right: pd.merge(left, right, on=['index','입력주소'], how='outer'), dfs)

df_merge['happy'] = 1
df_merge = df_merge[['입력주소','happy']]
address = df_merge.drop_duplicates().reset_index()

address.to_csv('address.csv', encoding='utf-8') # Geocoding 할 수 있도록 데이터 저장

### 공시지가 데이터에서 주택만 추출

- 2번 표제부 데이터에서 아파트, 단독주택 데이터만 추출하여 새로운 변수로 저장

In [ ]:
# 공시지가 데이터에서 입력주소, 주소 코드 만들어주기

data18['지번'] = data18['지번'].str.replace("_","-")  # 지번 바꾸기
data18['입력주소'] = data18['법정동명'] + " " + data18['지번'] + "번지"
data18['법정동코드'] = data18['법정동코드'].astype(str)
data18['code'] = data18['법정동코드'] + '/' + data18['지번']

In [ ]:
#표제부에서 주택(아파트, 단독) 에 해당하는 지번만 추출하기

data2['주용도코드명'] = data2['주용도코드명'].fillna('결측치')
house = data2[data2['주용도코드명'].str.contains('주택')] #표제부에서 주택만 추출
house['지'] = house['지'].astype(str)
house['번'] = house['번'].astype(str)
house['법정동코드'] = house['법정동코드'].astype(str)
house['시군구코드'] = house['시군구코드'].astype(str)

# 지번 변수 만들기
drop = house.loc[(house['지'] == '0') & (house['번'] == '0')].index
house = house.drop(drop)
house1 = house[house['지'] == '0']
house2 = house[house['번'] == '0']
house3 = house.loc[(house['지'] != '0') & (house['번'] != '0')]
house1['지번'] = house1['번']
house3['지번'] = house3['번'] + '-' + house3['지']

# Data Merging
house = pd.concat([house1,house3])
house = house[['대지위치','건물명','주용도코드명','시군구코드','법정동코드','지번','사용승인일']]
house['code'] = house['시군구코드']+ house['법정동코드'] + '/' + house['지번'] #코드 만들기
house = house[['대지위치','주용도코드명','건물명','code','사용승인일']]

In [ ]:
# 표제부 데이터와 공시지가 데이터 합쳐주기 

houseonly = pd.merge(data18, house, on = 'code') #주택만 추출된 데이터!
houseonly = houseonly[['건물명','법정동명','기준년도','공시지가','대지위치','주용도코드명','사용승인일']]
houseonly.to_csv('houseonly.csv', encoding='utf-8') #Geocoding 위해서 저장

## 2) Geocoding 된 데이터들 격자에 부여

- Geocoding Tool을 활용하여, 지번주소를 위경도 좌표 데이터로 변환하는 plus_grid 함수 정의

In [ ]:
# 위경도 데이터를 격자로 부여하는 함수 정의

def plus_grid(df, geo, X, Y, drop=True):
    data1= gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df[X], df[Y])) #X는 경도, Y는 위도
    merge_data = gpd.sjoin(data1, geo, op = "within", how = "right")
    merge_data.crs = geo.crs
    if drop:
        merge_data = merge_data.dropna()
    return merge_data

###  버스정류장 데이터

- 세종시 버스정류장 관련 외부 데이터를 활용하여, 하루 배차 회수 1 이하, 정보 누락된 버스정류장 NODE 제외.

- plus_grid 함수 사용하여, 그리드 별 개수 COUNT 후 세종시 버스 종류인 BRT, 광역, 지선, 간선, 마을 버스를 column으로 지정



In [ ]:
BUS = pd.read_csv("sejong_busnode.csv", encoding='utf-8') # 버스정류장 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

TIME=pd.read_csv("time.csv", encoding='utf-8') # 버스 종류별 배차 횟수 데이터버스 종류별 배차 횟수 데이터

BUS=BUS.dropna(how="all")

#'정류장번호 없음'은 삭제함.
index1=BUS[BUS['NODENO']=="정류장번호 없음"].index
BUS=BUS.drop(index1)
# 정류장 번호 '0'은 삭제함
index2=BUS[BUS['NODENO']=="0"].index
BUS=BUS.drop(index2)
# index 새롭게 지정
BUS=BUS.reset_index()

for i in range(0,len(TIME)): # 버스에 따라, 배차 횟수를 hap이라는 열에 지정.
    for j in range(0,len(BUS)) :
        if BUS['BUS_NUM'][j] == TIME['버스이름'][i]:
            BUS['hap'][j] = TIME['개수'][i]

# PCA를 통해, 지수를 생성할 것인데 이때의 변수를 버스 종류로 할 것이기에
# BUS 데이터에 빈 열을 생성해주었다.
BUS['BRT']=""
BUS['광역버스']=""
BUS['지선버스']=""
BUS['간선버스']=""
BUS['마을버스']=""

BRT=np.array(['B2','900','B5'],dtype=object)
광역버스=np.array(['1000','1001','1004','1005'],dtype=object)
지선버스=np.array(['11','12','13','52','53','61','201','203','221','300','910'],dtype=object)
간선버스=np.array(['430','991'],dtype=object)
마을버스=np.array(['16','17','31','32','33','35','62','63','64','65','66','661','67','69','691','71','72','73','74','75','76','81','82','83','84','85','86','91','92','93','94','95'],dtype=object)

for i in range(0,len(BUS)):
    if BUS["BUS_NUM"][i] in BRT:
        BUS['BRT'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 광역버스:
        BUS['광역버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 지선버스:
        BUS['지선버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 간선버스:
        BUS['간선버스'][i]=BUS['hap'][i]
    elif BUS["BUS_NUM"][i] in 마을버스:
        BUS['마을버스'][i]=BUS['hap'][i]
    else:
        print("잘못됨")

BUS=BUS[['NODENO','GPSLATI','GPSLONG','BRT','광역버스','지선버스','간선버스','마을버스']]

# gpslong 열에 ,가 한 데이터에 포함되어있어 삭제함.
BUS['GPSLONG']=BUS['GPSLONG'].astype(str)
BUS['GPSLONG'] = BUS['GPSLONG'].str.replace(',', '').astype('float64')

# groupby 함수 사용 위해 float으로 변환.
BUS['NODENO']=BUS['NODENO'].apply(pd.to_numeric)
BUS['GPSLATI']=BUS['GPSLATI'].apply(pd.to_numeric)
BUS['GPSLONG']=BUS['GPSLONG'].apply(pd.to_numeric)
BUS['BRT']=BUS['BRT'].apply(pd.to_numeric)
BUS['광역버스']=BUS['광역버스'].apply(pd.to_numeric)
BUS['지선버스']=BUS['지선버스'].apply(pd.to_numeric)
BUS['간선버스']=BUS['간선버스'].apply(pd.to_numeric)
BUS['마을버스']=BUS['마을버스'].apply(pd.to_numeric)

# 버스정류장번호(NODENO)별 
BUS1=BUS.groupby(["NODENO","GPSLATI","GPSLONG"])['BRT','광역버스','지선버스','간선버스','마을버스'].sum().reset_index()

# NODENO는 같지만, GPSLATI와 GPSLONG이 다른 데이터 존재.
BUS1=BUS1.drop_duplicates(['NODENO'], keep=False)

In [ ]:
bus = plus_grid(BUS1, geo, 'GPSLONG', 'GPSLATI')

### CCTV 데이터

In [ ]:
df = pd.read_csv('sejong_cctv.csv') # CCTV 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# CCTV 데이터에서 필요한 컬럼만 추출
df = df[['카메라대수','카메라화소수','위도','경도']]

cctv = plus_grid(df, geo, '경도', '위도')

### 가로등 데이터

In [ ]:
df = pd.read_csv('sejong_light.csv') # 가로등 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 가로등 데이터에서 필요한 컬럼만 추출
df = df[['가로등위치명','소재지지번주소','위도','경도']]

street_lights = plus_grid(df, geo, '경도', '위도')

### 파출소 데이터

In [ ]:
df = pd.read_csv('sejong_policeoffice.csv') # 파출소 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# Hospital 데이터에서 필요한 컬럼만 추출
df = df[['name','입력주소','X','Y']]

police_office = plus_grid(df, geo, 'X', 'Y')

### 학교 데이터

In [ ]:
df = pd.read_csv('sejong_school.csv') # 학교 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 학교 데이터에서 필요한 컬럼만 추출
df = df[['학교명','학교구분','주소','X','Y']]

school = plus_grid(df, geo, 'X', 'Y')

### 공원 데이터

In [ ]:
df = pd.read_csv('sejong_park.csv') #공원데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 공원 데이터에서 필요한 컬럼만 추출
df = df[['공원명','소재지지번주소','공원구분','위도','경도']]

parks = plus_grid(df, geo, '경도', '위도')

### 학원 데이터

In [ ]:
# 데이터를 불러옵니다
df = pd.read_csv('sejong_academy.csv') #학원데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# 학원 데이터에서 필요한 컬럼만 추출
academy1 = df[['학원주소','X','Y']]

# 상권 데이터에서 학원 데이터를 추출합니다
academy2 = data14[data14['상권업종중분류명'].str.contains('학원')]
academy2 = academy2.rename({'lon':'X', 'lat':'Y', '도로명주소':'학원주소'}, axis='columns')
academy_cat = ['학원(종합)','학원-입시','학원-외국어/어학','학원-어린이영어','태권도장', '피아노/바이올린/기타','합기도장','서예/서화/미술']
academy2 = academy2[academy2.상권업종소분류명.isin(academy_cat)][['학원주소','X', 'Y']]

# 두 데이터를 합쳐줍니다
df = pd.merge(academy1, academy2, on=['X','Y','학원주소'], how='outer')

academy = plus_grid(df, geo, 'X', 'Y')

### 편의점 데이터

In [ ]:
# 데이터를 불러옵니다
df = data14

# 편의점 데이터만 추출합니다
df = df[df['상권업종소분류명'].str.contains('편의점')]
df = df[['행정동명','도로명주소','lon','lat']]

# 편의점 데이터를 격자 데이터에 부여
stores = plus_grid(df, geo, 'lon', 'lat')

### 은행 데이터

In [ ]:
df = pd.read_csv('sejong_bank.csv') #은행데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['name', 'address', '경도','위도']]

# 은행 데이터를 격자 데이터에 부여
banks = plus_grid(df, geo, '경도', '위도')

### 약국 데이터

In [ ]:
df = pd.read_csv('sejong_pharmacy.csv') #약국데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['_약국명칭', '약국소재지 도로명주소', 'X','Y']]

# 약국 데이터를 격자 데이터에 부여
pharmacy = plus_grid(df, geo, 'X', 'Y')

### 마트 데이터

In [ ]:
df = pd.read_csv('sejong_market.csv') #약국데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['name', 'address', 'X','Y']]

# 약국 데이터를 격자 데이터에 부여
mart = plus_grid(df, geo, 'X', 'Y')

### 병원 데이터

In [ ]:
df = pd.read_csv('sejong_hospital.csv') # 병원 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

# Hospital 데이터에서 필요한 컬럼만 추출
df = df[['name','입력주소','X','Y']]

hospital= plus_grid(df, geo, 'X', 'Y')

### 매매 데이터

In [ ]:
# 평당 가격 구하는 함수 정의
def price_per(df , col='거래금액(만원)'):
    df['평'] = df['전용면적(㎡)']/3.3
    df['거래금액'] = df[col].str.replace(',','').astype(int)
    df['평당가격'] = df['거래금액']/df['평']
    df = df.drop(['평',col], axis=1)
    return df

In [ ]:
# 매매, 전월세 데이터 geocoding 된 데이터
adr = pd.read_csv('geoaddress.csv')
adr = adr[['입력주소','X','Y']]

In [ ]:
df3 = price_per(data3)
df4 = price_per(data4)
df6 = price_per(data6)

In [ ]:
df3 = pd.merge(get_address(df3)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_buy = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(df4)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_buy = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(df6)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_buy = plus_grid(df6, geo, 'X','Y', drop=False)

### 임대 데이터

In [ ]:
# 전월세 데이터를 전세와 월세로 구분합니다
# 1이 전세, 2가 월세

df71 = data7[data7['전월세구분'] == '전세']
df72 = data7[data7['전월세구분'] == '월세']
df81 = data8[data8['전월세구분'] == '전세']
df82 = data8[data8['전월세구분'] == '월세']
df101 = data10[data10['전월세구분'] == '전세']
df102 = data10[data10['전월세구분'] == '월세']

In [ ]:
df71 = price_per(df71, col='보증금(만원)')
df81 = price_per(df81, col='보증금(만원)')
df101 = price_per(df101, col='보증금(만원)')

In [ ]:
# 전세 데이터

df3 = pd.merge(get_address(df71)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_full = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(df81)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_full = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(df101)[['계약년월','평당가격','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_full = plus_grid(df6, geo, 'X','Y', drop=False)

In [ ]:
# 임대(전세+월세) 데이터

df3 = pd.merge(get_address(data7)[['계약년월','입력주소','시군구']], adr, on='입력주소') #아파트 매매 데이터에 위경도 추가
apt_rent = plus_grid(df3, geo, 'X','Y', drop=False)

df4 = pd.merge(get_address(data8)[['계약년월','입력주소','시군구']], adr, on='입력주소') #연립다세대 매매 데이터에 위경도 추가
vil_rent = plus_grid(df4, geo, 'X','Y', drop=False)

df6 = pd.merge(get_address(data10)[['계약년월','입력주소','시군구']], adr, on='입력주소') #오피스텔 매매 데이터에 위경도 추가
ops_rent = plus_grid(df6, geo, 'X','Y', drop=False)

### 공시지가 데이터

In [ ]:
df = pd.read_csv('zin18.csv') #Geocoding 데이터
df = pd.merge(houseonly, df, left_on = '대지위치', right_on = '지번주소') # 공시지가 데이터
geo = gpd.read_file('final_geo.geojson') #격자 데이터

df = df[['건물명', '지번주소','기준년도','공시지가', 'X','Y']]

# 공시지가 데이터를 격자 데이터에 부여
gongsi = plus_grid(df, geo, 'X', 'Y', drop=False)

### 표제부 주택 데이터

In [ ]:
# Geocoding된 주택 데이터
res = pd.concat([pd.read_csv('houseadd_file1.csv'), pd.read_csv('houseadd_file2.csv')])

# 주택 데이터를 격자 데이터에 부여
reals = plus_grid(res, geo, 'X', 'Y')

### 인구 데이터

In [ ]:
#거주인구정보 500미터 격자 파일과 합치기
grid_500 = geo[['id', 'geometry']]
merge19 = gpd.sjoin(grid_500, data19, op='intersects', how = 'left').reset_index()

# 중복 제거해주기
dropdu = merge19[['gid','201710_20대_거주인구수']].drop_duplicates().index
merge19 = merge19.loc[dropdu,:]
merge19_group = merge19.groupby('id').sum().reset_index()
population = merge19_group

- - -
# 2. Visualization

## 0) 함수정의

- plot_grid_map 함수: QGIS를 활용하여 만든 500X500격자와 주어진 데이터를 id(그리드)별로 통합하여 FisherJenks 이론을 바탕으로 세종특별자치시 지도 위에 단계(k)별로 그리는 함수


In [ ]:
import mapclassify as mc
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from matplotlib.colors import ListedColormap
import matplotlib
%config InlineBackend.figure_format='retina' #화질 좋게 해주기

import matplotlib.font_manager as fm
nanumr = fm.FontProperties(fname='NanumSquareOTFRegular.otf', size=18)
nanumb = fm.FontProperties(fname='NanumSquareOTFBold.otf', size=18)

In [ ]:
def plot_grid_map(df, col, k = 6, title = 'No name', mode = None, cls_dict = None, how = None,
                  annotation = False, percen = True, colors = 'Blues', c_mode = 'NaturalBreaks'):
    
    # 주어진 데이터를 적절히 단계별로 나누는 코드
    if mode == 'cont_classify':
        dfdf = df[df[col]>0]
        if c_mode == 'NaturalBreaks':
            quantiles = mc.NaturalBreaks(dfdf[col].dropna(), k = k)
        elif c_mode == 'FisherJenks':
            quantiles = mc.FisherJenks(dfdf[col].dropna(), k = k)
        df['cls_value'] = quantiles.find_bin(df[col]).astype('str')
        df.loc[df[col].isnull(), 'cls_value'] = 'No Data'
        df.loc[df[col]<0, 'cls_value'] = 'Minus'
        cmap = plt.cm.get_cmap(colors, k)
        cmap_list = [rgb2hex(cmap(i)) for i in range(cmap.N)]
        if len(np.where(df['cls_value'].unique() == 'Minus')[0]) != 0:
            cmap_list.append('#F78181')
        if len(np.where(df['cls_value'].unique() == 'No Data')[0]) != 0:
            cmap_list.append('#bdbdbd')
        cmap_with_grey = ListedColormap(cmap_list)
    if mode == 'cluster':
        k = len(df[col].unique())
        df[col].fillna(-2, inplace=True)
        df[col].astype('int')
        df['cls_value'] = df[col] + 1
        df.sort_values('cls_value')
        df.loc[df[col]<0, 'cls_value'] = 'No Data'
        cmap = plt.cm.get_cmap(colors, k)
        cmap_list = [rgb2hex(cmap(i)) for i in range(cmap.N)]
        if len(np.where(df['cls_value'].unique() == 'No Data')[0]) != 0:
            cmap_list.append('#bdbdbd')
        cmap_with_grey = ListedColormap(cmap_list)
    
        
    # plot 그리는 코드
    fig, ax = plt.subplots(figsize=(12, 10))
    df.plot(column='cls_value', edgecolor='k', cmap=cmap_with_grey,linewidth=0.05,
             legend=True, legend_kwds=dict(loc='upper right'),ax=ax)
    
    # 범례이름 바꾸는 코드
    if mode == 'cont_classify':
        legend_labels = ax.get_legend().get_texts()
        upper_bounds = quantiles.bins
        bounds = []
        for index, upper_bound in enumerate(upper_bounds):
            if index == 0:
                lower_bound = float(df.cls_value.min())
            else:
                lower_bound = float(upper_bounds[index-1])
            
            if percen:
                bound = '{}% - {}%'.format(round(lower_bound, 1), round(upper_bound, 1))
            else:
                bound = '{} - {}'.format(round(lower_bound, 2), round(upper_bound, 2))
            bounds.append(bound)
    if mode == 'cluster':
        if 'No Data' in list(df['cls_value'].unique()):
            legend_labels = ax.get_legend().get_texts()
            bounds = []
            for num in list(price_merge['cls_value'].unique())[0:-1]:
                bound = 'cluster {}'.format(round(num))
                bounds.append(bound)     
        else:
            legend_labels = ax.get_legend().get_texts()
            bounds = []
            for num in list(price_merge['cls_value'].unique()):
                bound = 'cluster {}'.format(num)
                bounds.append(bound)  
        
    # replace the numerical legend labels
    for bound, legend_label in zip(bounds, legend_labels):
        legend_label.set_text(bound)
        
    ax.axis('off')
    ax.set_title(title, fontdict={'fontsize': '25', 'fontweight' : '3'} , fontproperties=nanumr)
    fig.show()

- 색깔 팔레트: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

## 1) 시기별 이슈 확인

### 시계열로 거래 데이터 시각화

In [ ]:
# datetime 형태로 바꿔줍니다

df3, df4, df5, df6  = data3, data4, data5, data6
df7, df8, df9, df10  = data7, data8, data9, data10

df3['계약년월'] = pd.to_datetime(df3['계약년월'], format='%Y%m')
df4['계약년월'] = pd.to_datetime(df4['계약년월'], format='%Y%m')
df5['계약년월'] = pd.to_datetime(df5['계약년월'], format='%Y%m')
df6['계약년월'] = pd.to_datetime(df6['계약년월'], format='%Y%m')
df7['계약년월'] = pd.to_datetime(df7['계약년월'], format='%Y%m')
df8['계약년월'] = pd.to_datetime(df8['계약년월'], format='%Y%m')
df9['계약년월'] = pd.to_datetime(df9['계약년월'], format='%Y%m')
df10['계약년월'] = pd.to_datetime(df10['계약년월'], format='%Y%m')

In [ ]:
# 전월세 데이터를 전세와 월세로 구분합니다
# 1이 전세, 2가 월세

df71 = df7[df7['전월세구분'] == '전세']
df72 = df7[df7['전월세구분'] == '월세']
df81 = df8[df8['전월세구분'] == '전세']
df82 = df8[df8['전월세구분'] == '월세']
df91 = df9[df9['전월세구분'] == '전세']
df92 = df9[df9['전월세구분'] == '월세']
df101 = df7[df7['전월세구분'] == '전세']
df102 = df7[df7['전월세구분'] == '월세']

In [ ]:
# 월별로 계약 건수를 세주는 함수를 만듭니다

def counting(df):
    df = df.set_index('계약년월')
    df = df.groupby([pd.Grouper(freq='1M')]).count()
    return df

In [ ]:
# 하나의 데이터프레임으로 합쳐줍니다

df = DataFrame()
df['apt_m'] = counting(df3)['계약일']
df['vil_m'] = counting(df4)['계약일']
df['house_m'] = counting(df5)['계약일']
df['op_m'] = counting(df6)['계약일']
df['apt_j'] = counting(df71)['계약일']
df['vil_j'] = counting(df81)['계약일']
df['house_j'] = counting(df91)['계약일']
df['op_j'] = counting(df101)['계약일']
df['apt_w'] = counting(df72)['계약일']
df['vil_w'] = counting(df82)['계약일']
df['house_w'] = counting(df92)['계약일']
df['op_w'] = counting(df102)['계약일']

df = df.reset_index()

In [ ]:
df['apt'] = df['apt_m'] + df['apt_j'] + df['apt_w']
df['vil'] = df['vil_m'] + df['vil_j'] + df['vil_w']
df['house'] = df['house_m'] + df['house_j'] + df['house_w']
df['op'] = df['op_m'] + df['op_j'] + df['op_w']
df['buy'] = df['apt_m'] + df['vil_m'] + df['house_m'] + df['op_m']
df['jun'] = df['apt_j'] + df['vil_j'] + df['house_j'] + df['op_j']
df['wol'] = df['apt_w'] + df['vil_w'] + df['house_w'] + df['op_w']
df['all'] = df['apt'] + df['vil'] + df['house'] + df['op']
deal = df

### 세종시 실거래량 추이

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=deal['계약년월'],y=deal['all'], line = dict(color='#0B0B61', width=3)))
fig.update_layout(title='세종시 실거래량 추이', plot_bgcolor='#F8F7F1')
fig.show()

### 세종시 거주형태별 실거래량 추이

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=deal['계약년월'],y=deal['apt'], line = dict(color='#0B0B61', width=3), name='아파트'))
fig.add_trace(go.Scatter(x=deal['계약년월'],y=deal['vil'], line = dict(color='#404040', width=3), name='연립다세대'))
fig.add_trace(go.Scatter(x=deal['계약년월'],y=deal['house'], line = dict(color='#F3C706', width=3), name='단독다가구'))
fig.add_trace(go.Scatter(x=deal['계약년월'],y=deal['op'], line = dict(color='#0B6121', width=3), name='오피스텔'))
fig.update_layout(title='세종시 거주형태별 거래량 추이', plot_bgcolor='#F8F7F1')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()


### 세종시 거래형태별 실거래량 추이

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=df['계약년월'],y=df['buy'], line = dict(color='#0B0B61', width=3), name='매매'))
fig.add_trace(go.Scatter(x=df['계약년월'],y=df['jun'], line = dict(color='#0B6121', width=3), name='전세'))
fig.add_trace(go.Scatter(x=df['계약년월'],y=df['wol'], line = dict(color='#F3C706', width=3), name='월세'))
fig.update_layout(title='세종시 거래형태별 거래량 추이', plot_bgcolor='#F8F7F1')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

네이버 뉴스 크롤링 자료 활용 (2017년 1월 ~ 2020년 12월까지, '세종시 부동산' 이라는 키워드로 뉴스 기사 크롤링

In [ ]:
df2017 = pd.read_csv('sejong2017.csv', encoding= 'utf-8')
df2018 = pd.read_csv('sejong2018.csv', encoding= 'utf-8')
df2019 = pd.read_csv('sejong2019.csv', encoding= 'utf-8')
df2020 = pd.read_csv('sejong2020.csv', encoding= 'utf-8')

In [ ]:
# Data concat
df = pd.concat([df2017,df2018,df2019,df2020])

In [ ]:
# 데이터 길이 확인
print('2017년: ',len(df2017))
print('2018년: ',len(df2018))
print('2019년: ',len(df2019))
print('2020년: ',len(df2020))

### Tokenizing package

In [ ]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

### 데이터 전처리

In [ ]:
# 불용어 설정
stopwords = ['않다','에서','있다','없다','그렇다','아니다','것','이다','의','가','이','은','들',
             '는','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다','휴','수','세종시','세종','부동산']

In [ ]:
# 훈련 데이터 한글과 공백을 제외하고 모두 제거
df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['title'].replace('', np.nan, inplace=True)

### 뉴스 개수 변화 추이

In [ ]:
df['years'] = pd.to_datetime(df['years'], format='%Y.%m.%d.')

In [ ]:
def counting(df):
    df = df.set_index('years')
    df = df.groupby([pd.Grouper(freq='1M')]).count()
    return df

In [ ]:
df2 = counting(df)

In [ ]:
df2 = df2.drop(['contents','link','Unnamed: 5','Unnamed: 6','Unnamed: 7'],axis = 1)

In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.io as pio

fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=df2.index,y=df2['title'], line = dict(color='#0B0B61', width=3)))
fig.update_layout(title='연월별 뉴스 보도 개수 추이', plot_bgcolor='#F8F7F1')
fig.show()

### 기사 제목 토큰화

In [ ]:
okt = Okt()
token = []
for sentence in df['title']:
    temp_X = []
    temp_X = okt.nouns(sentence) # 명사 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    token.append(temp_X)
df['token'] = token
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token)

### 기준에 따른 분류 - 12개 구간으로 설정

In [ ]:
df = df.sort_values('years')

import datetime
df_1 = df[df['years'] <'2017-04-01'] # 3 / 1,2,3
df_2 = df[(df['years']>= '2017-04-01')&(df['years'] <'2017-07-01')] # 3 / 4,5,6
df_3 = df[(df['years']>= '2017-07-01')&(df['years'] <'2017-11-01')] # 4 / 7,8,9,10
df_4 = df[(df['years']>= '2017-11-01')&(df['years'] <'2018-01-01')] # 2 / 11,12
df_5 = df[(df['years']>= '2018-01-01')&(df['years'] <'2018-05-01')] # 4 / 1,2,3,4
df_6 = df[(df['years']>= '2018-05-01')&(df['years'] <'2019-02-01')] # 9
df_7 = df[(df['years']>= '2019-02-01')&(df['years'] <'2019-10-01')] # 8
df_8 = df[(df['years']>= '2019-10-01')&(df['years'] <'2020-01-01')] # 3
df_9 = df[(df['years']>= '2020-01-01')&(df['years'] <'2020-05-01')] # 4
df_10 = df[(df['years']>= '2020-05-01')&(df['years'] <'2020-08-01')] # 3
df_11 = df[(df['years']>= '2020-08-01')&(df['years'] <'2020-11-01')] # 3
df_12 = df[df['years']>= '2020-11-01'] # 2

In [ ]:
from collections import Counter # 단어 빈도 수 세기
def tokenizing(df):
    #konlpy로 명사만 추출하는 토큰화를 진행
    words = np.hstack(df['token'].values)
    word_count = Counter(words)
    #print(word_count.most_common(20))
    input = dict(word_count.most_common(300))
    return input

In [ ]:
df1 = tokenizing(df_1)
df2 = tokenizing(df_2)
df3 = tokenizing(df_3)
df4 = tokenizing(df_4)
df5 = tokenizing(df_5)
df6 = tokenizing(df_6)
df7 = tokenizing(df_7)
df8 = tokenizing(df_8)
df9 = tokenizing(df_9)
df10 = tokenizing(df_10)
df11 = tokenizing(df_11)
df12= tokenizing(df_12)

### 워드 클라우드 그리기

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [ ]:
# 워드클라우드를 그리는 함수 만들기
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

def wcdraw(inputs,width,height,path):
    pic = np.array(Image.open(path))
    image_colors = ImageColorGenerator(pic)
    # 네모 모양으로 wordcloud 생성하기
    wordcloud = WordCloud(font_path = 'NanumSquareOTFBold.otf', max_words=40,
                          width=width,height=height,background_color ='white',)

    # 워드 클라우드 그리기
    wordcloud = wordcloud.generate_from_frequencies(inputs)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation='bilinear')
    plt.axis("off")

In [ ]:
#세종시 지도 모양으로 자르기
pic = np.array(Image.open('sejong.png'))
wordcloud = WordCloud(font_path = 'NanumSquareOTFBold.otf', max_words=300, stopwords = '주택',
                        width=2000,height=2500,background_color ='white',colormap = 'ocean',mask = pic)

# 워드 클라우드 그리기
wordcloud = wordcloud.generate_from_frequencies(tokenizing(df))
plt.figure(figsize = (15 , 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# 2017년 1월 1일 ~ 3월 31일
wcdraw(df1,900,500,'1-1.png')

In [ ]:
# 2017년 4월 1일 ~ 7월 31일
wcdraw(df2,900,500,'1-2.png')

In [ ]:
# 2017년 7월 1일 ~ 10월 31일
wcdraw(df3, 1200,500, '1-3.png')

In [ ]:
#2017년 11월 ~ 2017년 12월 31일
wcdraw(df4,600,500,'1-4.png')

In [ ]:
#2018년 1월 ~ 4월 30일
wcdraw(df5, 600, 500, '2-1.png')

In [ ]:
# 2018년 5월 ~ 2019년 1월 
wcdraw(df6, 1350,500,'2-2.png')

In [ ]:
#2019년 2월 ~ 2019년 9월
wcdraw(df7, 1200, 500, '2-3.png')

In [ ]:
#2019년 10월 ~ 12월 31일
wcdraw(df8, 450, 500, '2-4.png')

In [ ]:
# 2020년 1월 ~ 4월
wcdraw(df9, 1200, 500, '3-1.png')

In [ ]:
# 2020년 5월 ~ 7월 31일
wcdraw(df10, 900, 500, '3-2.png')

In [ ]:
# 2020년 8월 ~ 10월 31일
wcdraw(df11, 900, 500, '3-3.png')

In [ ]:
# 2020년 11월 1일 ~ 12월 31일
wcdraw(df12, 600, 500, '3-4.png')

## 2) 주택 및 거래 관련

### 주택 나이 데이터

In [ ]:
# 주택 나이 나타내는 column 추가
df = pd.merge(house,res,on='대지위치',how='right')[['X','Y','사용승인일']]
df = df.dropna()
df['사용연도'] = df['사용승인일'].astype(str).str[:4]
df['사용연도'] = 2022 - df['사용연도'].astype(int)

# 그리드 별 평균 구해주기
df = plus_grid(df, geo, 'X', 'Y').groupby('id').mean()
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)
age = df[['id','사용연도']].fillna(0)

# 시각화
plot_grid_map(df, col = '사용연도', title = '건축 연한', k=10,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Purples', percen = False)

### 주택 매매 데이터

In [ ]:
from functools import reduce
col = ['id','계약년월','평당가격','geometry','입력주소']
dfs = [apt_buy[col],vil_buy[col],ops_buy[col]]
buy = reduce(lambda left, right: pd.merge(left, right, on=col, how='outer'), dfs)

In [ ]:
# 상승장 이전 이후로 나누기
before1 = buy.loc[buy['계약년월']<201910] #상승장 이전
after1 = buy[buy['계약년월']>201909] #상승장 이후

# 그리드 당 평균 매매 가격구하기
before = DataFrame(before1.groupby(['id']).mean()['평당가격']).reset_index()
after = DataFrame(after1.groupby(['id','계약년월']).mean()['평당가격']).reset_index()
all = DataFrame(buy.groupby(['id','계약년월']).mean()['평당가격']).reset_index()

# datetime으로 바꾸기
from datetime import datetime
after['계약년월'] = pd.to_datetime(after['계약년월'], format='%Y%m')
all['계약년월'] = pd.to_datetime(all['계약년월'], format='%Y%m')

In [ ]:
all.to_csv('grid_price.csv', encoding='utf-8')

In [ ]:
# 추이 간단히 시각화
plt.figure(figsize=(10,5))
for i in all['id'].unique().tolist():
    d_ = all[(all["id"]==i)]
    plt.plot(d_["계약년월"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('매매 가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

### 상승장 이전 매매 가격 평균

In [ ]:
df_before = pd.merge(before,geo,on='id',how='outer')
df_before = GeoDataFrame(df_before)

# 시각화
plot_grid_map(df_before, # 데이터: 격자 - 수치변수 이렇게 두개만 들어와 있는 GeoDataFrame 형태여야함
              col = '평당가격', # 수치변수 들어와있는 컬럼명
              title = '상승장 이전 매매가격 평균', # 시각화에 붙이고 싶은 제목 
              k =10, # 수치변수를 몇단계로 나누고 싶은지
              mode = 'cont_classify', #FisherJenks나 NaturalBreaks(KMeans) 쓸거면 얘 그냥 적으면 됨
              c_mode = 'FisherJenks', #케이민즈 쓰고 싶으면 NaturalBreaks 쓰기
              colors = 'Greens', #쓰고 싶은 색깔,, Blues Greens Purples 등등 다 가능함
              percen = False)

### 매매 거래 건수

In [ ]:
df = buy.dropna()
df = df.groupby('id').count().reset_index()[['id','평당가격']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '평당가격', title = '매매 거래 건수', k=10,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Blues', percen = False)

In [ ]:
df = before1.dropna()
df = df.groupby('id').count().reset_index()[['id','평당가격']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '평당가격', title = '상승장 이전 매매 거래 건수', k=10,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Blues', percen = False)

In [ ]:
df = after1.dropna()
df = df.groupby('id').count().reset_index()[['id','평당가격']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '평당가격', title = '상승장 이후 매매 거래 건수', k=10,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Blues', percen = False)

### 매매 실거래가 추이 Clustering

In [ ]:
#그리드 당 매매가격 불러오기
grid_price = all.copy()

#계약년월 열에 날짜 속성 부여 
grid_price_n = grid_price.set_index('계약년월')

#1년 단위로 자료 resampling
grid_price_y = grid_price_n.groupby('id')['평당가격'].resample('Y').mean().reset_index()
grid_price_y['year'] = grid_price_y['계약년월'].dt.year

df1 = grid_price_y.pivot(index='id', columns='year', values='평당가격').reset_index()

#결측치 3개 이상인 값 drop
df1.dropna(thresh=3, inplace=True)

#결측치 0으로 대체
df1.fillna(0, inplace=True)
value = df1.iloc[:,1:]

In [ ]:
#AgglomerativeClustering을 이용하여, 7개의 군집으로 clustering
cluster_price = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')
cluster_price.fit_predict(value)
df1["cluster"] = cluster_price.fit_predict(value)
df2 = pd.melt(df1, id_vars=['id'], value_vars=df1.iloc[:,1:-1], value_name='평당가격')
df3 = df1[['id', 'cluster']]
df4 = pd.merge(df2, df3, on = 'id', how = 'left')

#연도만 따로 추출한 year 열에 날짜 속성 부여
df4['year'] = pd.to_datetime(df4['year'], format='%Y')

In [ ]:
#군집 순서를 완만->급격 순으로 임의 조정
df4['cluster_n'] = df4['cluster'] + 10
df4['cluster_n'].replace(to_replace=[10,11,12,13,14,15,16], value= [0,2,6,3,1,5,4], inplace=True)
df4 = df4.drop(['cluster'], axis=1)
df4.rename(columns = {'cluster_n':'cluster'}, inplace=True)

#군집 순서를 완만->급격 순으로 임의 조정
df_price = df1[['id', 'cluster']]
df_price['cluster_n'] = df_price['cluster'] + 10
df_price['cluster_n'].replace(to_replace=[10,11,12,13,14,15,16], value= [0,2,6,3,1,5,4], inplace=True)
df_price = df_price.drop(['cluster'], axis=1)
df_price.rename(columns = {'cluster_n':'cluster'}, inplace=True)

### 클러스터 별 plot

In [ ]:
#군집1 plot
cluster_1 = df4[df4['cluster'] == 0]

plt.figure(figsize=(10,5))
for i in cluster_1['id'].unique().tolist():
    d_ = cluster_1[(cluster_1["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집1_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집2 plot
cluster_2 = df4[df4['cluster'] == 1]

plt.figure(figsize=(10,5))
for i in cluster_2['id'].unique().tolist():
    d_ = cluster_2[(cluster_2["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집2_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집3 plot
cluster_3 = df4[df4['cluster'] == 2]

plt.figure(figsize=(10,5))
for i in cluster_3['id'].unique().tolist():
    d_ = cluster_3[(cluster_3["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집3_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집4 plot
cluster_4 = df4[df4['cluster'] == 3]

plt.figure(figsize=(10,5))
for i in cluster_4['id'].unique().tolist():
    d_ = cluster_4[(cluster_4["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집4_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집5 plot
cluster_5 = df4[df4['cluster'] == 4]

plt.figure(figsize=(10,5))
for i in cluster_5['id'].unique().tolist():
    d_ = cluster_5[(cluster_5["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집5_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집6 plot
cluster_6 = df4[df4['cluster'] == 5]

plt.figure(figsize=(10,5))
for i in cluster_6['id'].unique().tolist():
    d_ = cluster_6[(cluster_6["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집6_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#군집7 plot
cluster_7 = df4[df4['cluster'] == 6]

plt.figure(figsize=(10,5))
for i in cluster_7['id'].unique().tolist():
    d_ = cluster_7[(cluster_7["id"]==i)]
    plt.plot(d_["year"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('군집7_매매가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#격자 데이터와 병합
price_merge = pd.merge(df_price, geo, on = 'id', how = 'right')

#cluster 순서에 따라 데이터 정렬
price_merge.sort_values('cluster', inplace=True)

#geodataframe으로 변환
price_merge = gpd.GeoDataFrame(price_merge)

In [ ]:
# 시각화
plot_grid_map(price_merge, col = 'cluster', title = '평당가격 클러스터링', mode = 'cluster',
              c_mode = 'NaturalBreaks', colors = 'Blues', percen = False)

### 주택 전세 데이터

In [ ]:
from functools import reduce
col = ['id','계약년월','평당가격','geometry','입력주소']
dfs = [apt_full[col],vil_full[col],ops_full[col]]
full = reduce(lambda left, right: pd.merge(left, right, on=col, how='outer'), dfs)

In [ ]:
# 그리드 당 평균 전세 가격구하기
df = DataFrame(full.groupby(['id','계약년월']).mean()['평당가격']).reset_index()

# datetime으로 바꾸기
from datetime import datetime
df['계약년월'] = pd.to_datetime(df['계약년월'], format='%Y%m')

# 추이 간단히 시각화
plt.figure(figsize=(10,5))
for i in df['id'].unique().tolist():
    d_ = df[(df["id"]==i)]
    plt.plot(d_["계약년월"], d_["평당가격"], "-", alpha=.6)
plt.grid()
plt.title('전세 가격 추이', fontproperties=nanumr)
plt.xticks(rotation=90)
plt.show()

### 주택 임대 데이터

In [ ]:
from functools import reduce
col = ['id','계약년월', 'geometry','입력주소']
dfs = [apt_rent[col],vil_rent[col],ops_rent[col]]
rent = reduce(lambda left, right: pd.merge(left, right, on=col, how='outer'), dfs)
before2 = rent[rent['계약년월']<201910] #상승장 이전
after2 = rent[rent['계약년월']>201909] #상승장 이후

In [ ]:
# 그리드 당 평균 임대 건수구하기
df = DataFrame(rent.groupby(['id','계약년월']).count()['입력주소']).reset_index()

# datetime으로 바꾸기
from datetime import datetime
df['계약년월'] = pd.to_datetime(df['계약년월'], format='%Y%m')

# 추이 간단히 시각화
plt.figure(figsize=(10,5))
for i in df['id'].unique().tolist():
    d_ = df[(df["id"]==i)]
    plt.plot(d_["계약년월"], d_["입력주소"], "-", alpha=.6)
plt.grid()
plt.legend(fontsize=13)
plt.xticks(rotation=90)
plt.show()

### 임대 거래 건수

In [ ]:
df = rent.dropna()
df = df.groupby('id').count().reset_index()[['id','입력주소']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '입력주소', title = '임대 거래 건수', mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'Blues', percen = False)

In [ ]:
df = before2.dropna()
df = df.groupby('id').count().reset_index()[['id','입력주소']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '입력주소', title = '상승장 이전 임대 거래 건수', mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'Blues', percen = False)

In [ ]:
df = after2.dropna()
df = df.groupby('id').count().reset_index()[['id','입력주소']]
df = pd.merge(df,geo,on='id',how='outer')
df = GeoDataFrame(df)

# 시각화
plot_grid_map(df, col = '입력주소', title = '상승장 이후 임대 거래 건수', mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'Blues', percen = False)

### 행정동별 전세가율 추이

In [ ]:
# 아파트 데이터만 불러오기
aptb = apt_buy.dropna()
aptf = apt_full.dropna()
aptb['시군구'] = aptb['시군구'].str[8:11]
aptf['시군구'] = aptf['시군구'].str[8:11]

# 날짜 별로 정렬
aptf = DataFrame(aptf.groupby(['시군구','입력주소','계약년월']).mean()['평당가격']).reset_index()
aptb = DataFrame(aptb.groupby(['시군구','입력주소','계약년월']).mean()['평당가격']).reset_index()

df2 = pd.merge(aptb, aptf, on=['시군구','입력주소', '계약년월'], how='outer')
df2['전세가율'] = 100*df2['평당가격_y']/df2['평당가격_x']
df2['계약년월'] = pd.to_datetime(df2['계약년월'], format='%Y%m')

df2 = df2.dropna().groupby(['시군구', '계약년월']).mean().unstack()['전세가율'].transpose()

# 시각화
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
for i in df2.columns.tolist():
    fig.add_trace(go.Scatter(x=df2.index,y=df2[i],
             mode='lines', name=i))
fig.update_layout(title='세종시 전세가율 추이', plot_bgcolor='#F8F7F1')
fig.show()

In [ ]:
# Data Merging
df2 = DataFrame(df2.mean()).reset_index()
df2 = df2.replace('조치원','조치원읍')
df2 = pd.merge(df2,data31,left_on='시군구',right_on='EMD_KOR_NM',how='outer')
df2 = GeoDataFrame(df2)

# 시각화
plot_grid_map(df2, col = 0, title = '전세가율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'Blues', percen = True)

### 공시지가 변동률

In [ ]:
# 연도별 공시지가 변동률 구하기

df = gongsi.groupby(['지번주소','id','기준년도']).mean()['공시지가'].unstack()
df = df.dropna(thresh=2)
df = df.reset_index()

# 변동률이기에 로그 근사 시켜주기

df['17-18'] = (df[2018.0] - df[2017.0])/df[2017.0] +1
df['18-19'] = (df[2019.0] - df[2018.0])/df[2018.0] +1
df['19-20'] = (df[2020.0] - df[2019.0])/df[2019.0] +1

df[['17-18','18-19','19-20']] = np.log(df[['17-18','18-19','19-20']])
df[[2017.0, 2018.0, 2019.0, 2020.0]] = df[[2017.0, 2018.0, 2019.0, 2020.0]]*3.3/10000
df.fillna(0)
df['변동률'] = (df['17-18'] + df['18-19'] + df['19-20'])*100/3
gongsi2 = df.groupby('id').mean().reset_index()

In [ ]:
gongsi2 = pd.merge(gongsi2,geo,on='id',how='outer')
df2 = GeoDataFrame(gongsi2)

# 시각화
plot_grid_map(df2, col = '변동률', title = '공시지가 변동률',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'BuGn', percen = True)

### 2020년 공시지가

In [ ]:
# 시각화
plot_grid_map(df2, col = 2020.0, title = '2020년 공시지가',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'BuGn', percen = False)

## 3) 인구 관련

In [ ]:
df = population.set_index('id')

### 청년층 비율

In [ ]:
# 2020년 데이터만 추출

df2020 = df[['202010_20대_거주인구수','202010_30대_거주인구수','202010_40대_거주인구수',
        '202010_50대_거주인구수','202010_60대_거주인구수','202010_70대_거주인구수',
        '202010_80대_거주인구수','202010_90대_거주인구수','202010_100세이상_거주인구수']].fillna(0)

# 청년층 비율 계산
df2020['총합'] = df2020.sum(axis=1)
df2020['청년층비율'] = 100*(df2020['202010_20대_거주인구수']+df2020['202010_30대_거주인구수'])/df2020['총합']
df2020['노년층비율'] = 100*(df2020['202010_70대_거주인구수']+df2020['202010_80대_거주인구수']+
                       df2020['202010_90대_거주인구수']+df2020['202010_100세이상_거주인구수'])/df2020['총합']
df2020 = df2020.reset_index()

# 시각화
df2020 = pd.merge(df2020,geo,on='id',how='outer')
df2020 = GeoDataFrame(df2020)
plot_grid_map(df2020, col = '청년층비율' , title = '청년층',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

In [ ]:
pop2020 = df2020

### 노년층 비율

In [ ]:
# 시각화
plot_grid_map(df2020, col = '노년층비율' , title = '노년층',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 전입자 수, 전출자 수 데이터

In [ ]:
# 전입자 수 데이터
dfin = data20.groupby(['년월','세종전입행정동']).sum().unstack()
dfin.columns = dfin.columns.droplevel(0)
dfin = dfin.rename_axis(None, axis=1).fillna(0)

# 전출자 수 데이터
dfout = data21.groupby(['년월','세종전출행정동']).sum().unstack()
dfout.columns = dfout.columns.droplevel(0)
dfout = dfout.rename_axis(None, axis=1).fillna(0)

# 인덱스 datetime 으로 변환
dfin.index = pd.to_datetime(dfin.index, format='%Y%m')
dfout.index = pd.to_datetime(dfout.index, format='%Y%m')

In [ ]:
#동별 순 전입자수 시각화
from plotly.subplots import make_subplots

df= dfin - dfout
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
for i in df.columns.tolist():
    fig.add_trace(go.Scatter(x=df.index,y=df[i],
             mode='lines', name=i))
fig.update_layout(title='순 유입(전입자 수 - 전출자 수)', plot_bgcolor='#F8F7F1')
fig.show()

### 내지인 거래 비율

In [ ]:
# 전출 동네가 세종시인 비율 구하기
data20['전출 동네'] = data20['전출시군구'] + ' ' + data20['전출읍면동']
df = data20.groupby(['세종전입행정동','전출시군구']).sum()['전입자수'].unstack().transpose().fillna(0)
df2 = df.transpose()
df2['세종비율'] = 100*df.loc['세종특별자치시']/df.sum()
df = DataFrame(df2['세종비율']).reset_index()

# Data Merging
df2 = pd.merge(df,data32,left_on='세종전입행정동',right_on='ADM_DR_NM',how='outer')
df2 = GeoDataFrame(df2)

# 시각화
plot_grid_map(df2, col = '세종비율', title = '세종시 전입 비율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 전출시도별 전입자수 추이

In [ ]:
# 전출시도별 전입자수 추이
data20_2 = data20.groupby(["년월","전출시"]).count().reset_index()
data20_2['년월'] = pd.to_datetime(data20_2['년월'], format='%Y%m')

table = data20_2.pivot_table(index = ['년월'],columns=['전출시'])
provinces = data20_2['전출시'].unique()

fig = make_subplots(rows=6, cols=3,horizontal_spacing=0.05,subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))

for i, province in enumerate(provinces):
    row, col, legend = i//3 + 1, i%3 + 1, False
    if i == len(provinces)-1 :
        legend = True
    fig.add_trace(go.Scatter(x=table.index,y= table['전입자수'][province],
                 mode='lines', name=col,showlegend=False), row=row, col=col)

fig.update_layout(title='<b>시도별 세종시 전입자수 추이</b>',
                  height = 2000,
                  legend=dict(x=0.7, y=0.05, traceorder="normal",
                             font=dict(family="sans-serif", size=18)), plot_bgcolor='#F8F7F1')    
for i in fig['layout']['annotations']:
    i['font'] = dict(size=9)

fig.show()

In [ ]:
data20_2 = data20_2.groupby('년월').sum().reset_index()
fig = px.bar(data20_2,x='년월',y = '전입자수', color_discrete_sequence=['navy'])
fig.update_layout(title='<b>전출시도별 전입자수 추이</b>', plot_bgcolor='#F8F7F1')
fig.show()

In [ ]:
# 순 전입자 수 구하기
df = dfin - dfout
df = DataFrame(df.sum()).reset_index()

# Data Merging
df2 = pd.merge(df,data32,left_on='index',right_on='ADM_DR_NM',how='outer')
df2 = GeoDataFrame(df2)
popin = df2

# 시각화
plot_grid_map(df2, col = 0 , title = '세종시 순전입자 수',mode = 'cont_classify', k=3,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = False)

In [ ]:
# 전입/전출 비율 구하기
df = 100*dfin.sum()/dfout.sum()
df = DataFrame(df).reset_index()

# Data Merging
df2 = pd.merge(df,data32,left_on='index',right_on='ADM_DR_NM',how='outer')
df2 = GeoDataFrame(df2)

# 시각화
plot_grid_map(df2, col = 0 , title = '세종시 전입 전출 비율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 읍면동 별 인구 수

In [ ]:
df = data22[['읍면동','202012_남자(총_거주자_수)','202012_여자(총_거주자_수)']]
emd = []
for i in range(len(df)):
    if df['읍면동'][i][2] == '면':
        emd.append('면')
    elif df['읍면동'][i][2] == '원':
        emd.append('읍')
        
    elif df['읍면동'][i][2] == '동':
        emd.append('동')
    else:
        emd.append('세종')
df['emd'] = emd
df['계'] = df['202012_남자(총_거주자_수)']+df['202012_여자(총_거주자_수)']

In [ ]:
dfdf = df.drop(0)
dfdf = dfdf.groupby('emd').sum().reset_index()
labels = dfdf['emd']
values = dfdf['계']

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, marker_colors=['#F5D042', '#0A174E', '#09663D'])])
fig.update_layout(legend=dict(xanchor="left",x=0.65),title='세종시 인구 수 비율',font=dict(size=18))
fig.show()

In [ ]:
labels = ['남자', '여자']
values = [176998, 177707]

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, marker_colors=['b', 'r'])])
fig.update_layout(legend=dict(xanchor="left",x=0.65),title='세종시 인구 수 비율',font=dict(size=18))
fig.show()

### 읍면동 별 가구 수

In [ ]:
# 읍면동으로 나누기
df = data28
df['읍면동'] = df['읍면동'].str.replace(" ", "") # 공백 제거

emd = []
for i in range(len(df)):
    if df['읍면동'][i][2] == '면':
        emd.append('면')
    elif df['읍면동'][i][2] == '원':
        emd.append('읍')
    elif df['읍면동'][i][2] == '동':
        emd.append('동')
df['emd'] = emd

In [ ]:
import plotly.graph_objects as go

dfdf = df.groupby('emd').sum().reset_index()
labels = dfdf['emd']
values = dfdf['계']

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, marker_colors=['#F5D042', '#0A174E', '#09663D'])])
fig.update_layout(legend=dict(xanchor="left",x=0.65), title='세종시 가구 수 비율',font=dict(size=18))
fig.show()

### 1인 가구 비율

In [ ]:
df = data28
df['읍면동'] = df['읍면동'].str.replace(" ", "") # 공백 제거

# 1인가구 비율 구하기
df['1인가구 비율'] = 100*df['1인']/df['계']

# 핵가족
df['핵가족 비율'] = 100*(df['2인']+df['3인']+df['5인'])/df['계']

# 대가족
df['대가족 비율'] = 100*(df['6인']+df['7인']+df['8인']+df['9인']+df['10인이상'])/df['계']

# Data Merging
df2 = pd.merge(df,data32,left_on='읍면동',right_on='ADM_DR_NM',how='outer')
df2 = GeoDataFrame(df2)

# 시각화
plot_grid_map(df2, col = '1인가구 비율' , title = '1인 가구 비율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 핵가족 가구 비율

In [ ]:
# 시각화
plot_grid_map(df2, col = '핵가족 비율' , title = '핵가족 가구 비율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 대가족 가구 비율

In [ ]:
# 시각화
plot_grid_map(df2, col = '대가족 비율' , title = '대가족 가구 비율',mode = 'cont_classify', k=10,
              c_mode = 'FisherJenks', colors = 'YlGn', percen = True)

### 거주 의사 데이터

In [ ]:
dat29 = data29.copy()
dat29['긍정'] =  data29['매우 그런편임']+ data29['어느정도 그런편임']
dat29['부정'] =  data29['약간 그렇지 않은편임']+ data29['전혀 그렇지 않은편임']
dat29 = dat29.drop(['매우 그런편임','어느정도 그런편임','전혀 그렇지 않은편임','약간 그렇지 않은편임'],axis =1)

In [ ]:
# 데이터, 변수별 분리
data29_dur = dat29.loc[6:7] # 거주기간별
data29_mar = dat29.loc[25:27] # 결혼여부별 
data29_hom = dat29.loc[38:40] # 주거점유형태별

### 거주기간별 향후 거주 의사 

In [ ]:
df = data29_dur.transpose().reset_index().drop([0,1])
for i in range(2):
    # Use `hole` to create a donut-like pie chart
    fig = go.Figure(data=[go.Pie(labels=df['index'], values=df[i+6], hole=.4, marker_colors=['#F5D042', '#0A174E', '#09663D'])])
    fig.update_layout(legend=dict(xanchor="left",x=0.65), title='향후 거주 의사',font=dict(size=18))
    fig.show()

### 주거점유형태별 향후 거주 의사 

In [ ]:
df = data29_hom.transpose().reset_index().drop([0,1])
for i in range(3):
    # Use `hole` to create a donut-like pie chart
    fig = go.Figure(data=[go.Pie(labels=df['index'], values=df[i+38], hole=.4, marker_colors=['#F5D042', '#0A174E', '#09663D'])])
    fig.update_layout(legend=dict(xanchor="left",x=0.65), title='향후 거주 의사',font=dict(size=18))
    fig.show()

- - -
# 3. Modeling

## 1) 지수 생성

## Clustering

### 클러스터링 함수 정의

In [ ]:
# pydeck 용 coordinates 만들어주기
def multipolygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

data32['coordinates'] = data32['geometry'].apply(multipolygon_to_coordinates)

In [ ]:
# 시각화 함수
def clustermap(df, X, Y):
    layer1 = pdk.Layer('ScatterplotLayer', # 사용할 Layer 타입
                  df, # 시각화에 쓰일 데이터프레임
                  get_position=[X, Y], # geometry 정보를 담고있는 컬럼 이름
                  get_color='색상', # 각 데이터 별 rgb 또는 rgba 값 (0~255) # 마지막이 투명도 (0~255사이)
                  get_radius=100
                  #wireframe=True, # 마우스 오버(hover) 시 박스 출력 
                  ) # 3D의 경우 값의 높이를 val 변수를 통해 하겠다.

    layer2 = pdk.Layer('PolygonLayer', 
                       data32, 
                       get_polygon='coordinates', 
                       get_fill_color='[100,150,200,30]',
                       get_line_color='[255, 255, 255]',
                       pickable=True, 
                       lineWidthScale=15,
                       auto_highlight=True)

    center = [127.261575, 36.562811]
    view_state = pdk.ViewState(
        longitude=center[0],
        latitude=center[1],
        zoom=9.7)

    r = pdk.Deck(layers=[layer1,layer2], initial_view_state=view_state,
                 mapbox_key ='pk.eyJ1Ijoia3Nra253IiwiYSI6ImNra2NyejlqNzAzamsyb3BtbndvMnVvZjYifQ.xGhIzKCQe1mfade6s__fNw', 
                 map_style='mapbox://styles/mapbox/dark-v9')

    return r

### 상권 클러스터링

In [ ]:
# HDBSCAN으로 상권 클러스터링
clusterer = hdbscan.HDBSCAN(cluster_selection_epsilon=0.0022)
df = data14
clusterer.fit(df[['lon', 'lat']])
df['cluster'] = clusterer.labels_
print(len(df['cluster'].unique()))
print(len(df[df['cluster'] == -1]))
df = df[df['cluster'] != -1]

In [ ]:
색상 = []
for kind in df['cluster']:
    np.random.seed(kind + 10000)
    
    색상_temp = np.random.randint(250, size=(3, )).tolist()
    if kind == -1:
        색상_temp.extend([0])
    else:
        색상_temp.extend([170])
    
    색상.append(색상_temp)
    
df['색상'] = 색상
df = df.reset_index(drop=True)

In [ ]:
clustermap(df, 'lon','lat').to_html()

In [ ]:
drop = df.groupby('cluster').count()['lon'][df.groupby('cluster').count()['lon'] <= 25].index
clustermap(df[~df['cluster'].isin(drop)], 'lon','lat').to_html()

In [ ]:
print('제거 전: ', len(df['cluster'].unique()))
print('제거 후: ', len(df[~df['cluster'].isin(drop)]['cluster'].unique()))

In [ ]:
store_delete = df[~df['cluster'].isin(drop)]
store_delete = gpd.GeoDataFrame(store_delete, geometry= gpd.points_from_xy(store_delete.lon, store_delete.lat))

In [ ]:
#같은 군집에 속하는 좌표들의 합집합을 구하고, 그 합집합의 중점 구하기
aca_cluster = {"cluster":[],"lon":[], "lat":[]}

for i in list(store_delete.cluster.unique()):
    aca_cluster["cluster"].append(i)
    aca_cluster["lon"].append(store_delete[store_delete.cluster == i].unary_union.centroid.x)
    aca_cluster["lat"].append(store_delete[store_delete.cluster == i].unary_union.centroid.y)
    
store_cluster_df = pd.DataFrame(aca_cluster)
store_cluster_geo = gpd.GeoDataFrame(store_cluster_df, geometry = gpd.points_from_xy(store_cluster_df.lon,store_cluster_df.lat))
store_cluster_geo.crs = geo.crs

### 학원가 클러스터링

In [ ]:
# HDBSCAN으로 학원가 클러스터링
clusterer = hdbscan.HDBSCAN(cluster_selection_epsilon=0.003)
df = academy 
clusterer.fit(df[['X', 'Y']])
df['cluster'] = clusterer.labels_
print(len(df['cluster'].unique()))
print(len(df[df['cluster'] == -1]))
df = df[df['cluster'] != -1]

In [ ]:
색상 = []
for kind in df['cluster']:
    np.random.seed(kind + 10000)
    
    색상_temp = np.random.randint(250, size=(3, )).tolist()
    if kind == -1:
        색상_temp.extend([0])
    else:
        색상_temp.extend([170])
    
    색상.append(색상_temp)
    
df['색상'] = 색상
df = df.reset_index(drop=True)

In [ ]:
clustermap(df, 'X','Y').to_html()

In [ ]:
drop = df.groupby('cluster').count()['X'][df.groupby('cluster').count()['X'] <= 20].index
clustermap(df[~df['cluster'].isin(drop)], 'X','Y').to_html()

In [ ]:
print('제거 전: ', len(df['cluster'].unique()))
print('제거 후: ', len(df[~df['cluster'].isin(drop)]['cluster'].unique()))

In [ ]:
academy_delete = df[~df['cluster'].isin(drop)]
academy_delete = gpd.GeoDataFrame(academy_delete, geometry= gpd.points_from_xy(academy_delete.X, academy_delete.Y))

In [ ]:
#같은 군집에 속하는 좌표들의 합집합을 구하고, 그 합집합의 중점 구하기
aca_cluster = {"cluster":[],"lon":[], "lat":[]}

for i in list(academy_delete.cluster.unique()):
    aca_cluster["cluster"].append(i)
    aca_cluster["lon"].append(academy_delete[academy_delete.cluster == i].unary_union.centroid.x)
    aca_cluster["lat"].append(academy_delete[academy_delete.cluster == i].unary_union.centroid.y)
    
academy_cluster_df = pd.DataFrame(aca_cluster)
academy_cluster_geo = gpd.GeoDataFrame(academy_cluster_df, geometry = gpd.points_from_xy(academy_cluster_df.lon,academy_cluster_df.lat))
academy_cluster_geo.crs = geo.crs

## 지수 생성용 전처리

In [ ]:
# 그리드 별 개수 구해서 추가하는 함수 정의
def grid_count(df, name):
    df = df.groupby('id').count().reset_index()
    df = df[['id','SIG_KOR_NM']]
    df.columns = ['id', name]
    return df

# 그리드 별 평균 구해서 추가하는 함수 정의
def grid_mean(df, name):
    df = df.groupby('id').mean().reset_index()
    df = df[['id','SIG_KOR_NM']]
    df.columns = ['id', name]
    return df

### 안전 지수

In [ ]:
# 그리드 별 가로등 개수 구하기
df_light = grid_count(street_lights, '가로등')

# 그리드 별 파출소 개수 구하기
df_police_office = grid_count(police_office, '파출소')

# 그리드 별 cctv 개수 구하기
df_cctv = cctv.groupby('id').sum().reset_index()[['id','카메라대수']]
df_cctv['카메라대수'] = df_cctv['카메라대수']*9

In [ ]:
from functools import reduce
dfs = [geo, df_light, df_police_office, df_cctv]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), dfs)
df_merge = df_merge.drop(['left','top','right','bottom','SIG_CD','SIG_KOR_NM'],axis=1)
safety = df_merge

### 교통 지수

역, 터미널과의 거리의 경우에는,  
- 오송역  
- 반석역  
- 조치원역  
- 세종고속시외버스터미널  
- 조치원공영버스터미널   
을 고려함

제외한 역  
- 전의역, 부강역도 하루에 몇 번 정차하지만 이용하기 불편한 횟수  
- 전동역, 서창역, 내판역, 매포역, 소정리역은 모든 여객열차가 정차하지 않음  
- 청주역: 충북이기도 하고, 사람들이 엄청 다니는 역도 아님.  

In [ ]:
# 그리드 별 버스 별 배차 개수 구하기
df_bus = bus[['id','BRT','광역버스','지선버스','간선버스','마을버스']]
df_bus=df_bus.groupby(["id"])['BRT','광역버스','지선버스','간선버스','마을버스'].sum().reset_index()

#  그리드 별 중앙 좌표와 가까운 역, 터미널과의 거리 구하기
station = pd.read_csv('sejong_stations.csv')
station_geo= gpd.GeoDataFrame(station, geometry = gpd.points_from_xy(station.경도, station.위도))
station_geo.crs = geo.crs

distance_station = []

for i in geo['geometry']:
    temp = []
    for j in station_geo['geometry']:
        temp.append(i.distance(j))
    
    distance_station.append(min(temp)) 
    
station_dist = geo.copy()
station_dist["역 터미널"] = distance_station
station_dist = station_dist[['id', '역 터미널']]

In [ ]:
from functools import reduce
dfs = [geo, df_bus, station_dist]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), dfs)
df_merge = df_merge.drop(['left','top','right','bottom','SIG_CD','SIG_KOR_NM'],axis=1)

In [ ]:
# 거리가 멀수록 페널티를 주기 위해 Hyperbolic Tangent 도함수에 넣어준 뒤, 정규화를 진행합니다
def diff_tanh(x):
    return 4/(np.exp(2*x)+2+np.exp(-2*x))

dfdf = diff_tanh(df_merge['역 터미널'])

dfdf = pd.DataFrame(dfdf, columns=['역 터미널'])
df_merge['역 터미널']=dfdf['역 터미널']
transport=df_merge

### 생활 편의 지수

In [ ]:
# 그리드 별 병원 개수 구하기
df_hospital = grid_count(hospital, '병원')

# 그리드 별 은행 개수 구하기
df_banks = grid_count(banks, '은행')

# 그리드 별 학교 개수 구하기
df_schools = school.groupby(['id','학교구분']).count().unstack()
df_schools = df_schools[df_schools.columns[0:3]].reset_index()
df_schools.columns = ['id','고등학교', '중학교', '초등학교']

# 그리드 별 공원 개수 구하기
df_parks = grid_count(parks, '공원')

# 그리드 별 학원 개수 구하기
df_academy = grid_count(academy, '학원')

# 그리드 별 편의점 개수 구하기
df_stores = grid_count(stores, '편의점')

# 그리드 별 마트 개수 구하기
df_mart = grid_count(mart, '마트')

# 그리드 별 약국 개수 구하기
df_pharmacy = grid_count(pharmacy, '약국')

# 그리드 별 중앙 좌표와 가까운 상권과의 거리 구하기
distance_store = []
for i in geo['geometry']:
    temp = []
    for j in store_cluster_geo['geometry']:
        temp.append(i.distance(j))
    
    distance_store.append(min(temp))
store_dist = geo.copy()
store_dist["상권"] = distance_store
store_dist = store_dist[['id', '상권']]


# 그리드 별 중앙 좌표와 가까운 학원가와의 거리 구하기
distance_academy = []
for i in geo['geometry']:
    temp = []
    for j in academy_cluster_geo['geometry']:
        temp.append(i.distance(j))
    
    distance_academy.append(min(temp)) 
academy_dist = geo.copy()
academy_dist["학원가"] = distance_academy
academy_dist = academy_dist[['id', '학원가']]

In [ ]:
from functools import reduce
dfs = [geo, df_hospital, df_banks, df_schools, df_parks, df_academy, 
       df_stores, df_pharmacy, df_mart, store_dist, academy_dist]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), dfs)
df_merge = df_merge.drop(['left','top','right','bottom','SIG_CD','SIG_KOR_NM'],axis=1)

In [ ]:
# 거리가 멀수록 페널티를 주기 위해 Hyperbolic Tangent 도함수에 넣어준 뒤, 정규화를 진행합니다
def diff_tanh(x):
    return 4/(np.exp(2*x)+2+np.exp(-2*x))

dfdf = diff_tanh(df_merge[['상권','학원가']])
df_merge[['상권','학원가']]=dfdf[['상권','학원가']]
life = df_merge

## PCA

- 여러 변수 간 존재하는 상관관계를 이용, 이를 대표하는 주성분을 추출하여 차원 축소하는 방법

- 각각 요소의 특징을 가장 잘 대변하는 단일 종합 지수 생성을 위해 PCA 선택

### 생활지수 

- 은행, 초등학교, 중학교, 고등학교, 공원, 학원, 편의점, 약국, 마트, 병원, 그리드별 가까운 상권/학원가와의 직선거리

In [ ]:
# NA 값은 0개이기에, 0으로 입력해준다.
life['은행']=life['은행'].fillna(0)
life['고등학교']=life['고등학교'].fillna(0)
life['중학교']=life['중학교'].fillna(0)
life['초등학교']=life['초등학교'].fillna(0) 
life['공원']=life['공원'].fillna(0)
life['학원']=life['학원'].fillna(0)
life['편의점']=life['편의점'].fillna(0) 
life['약국']=life['약국'].fillna(0)
life['마트']=life['마트'].fillna(0)
life['병원']=life['병원'].fillna(0)

In [ ]:
# PCA 후, plot_grid_map을 하기 위해 ID가 필요하므로 미리 추출한다.
id=life['id']

# PCA를 위해 x 설정.
x=life[['은행','고등학교','중학교','초등학교','공원','학원','편의점','약국','마트','상권','학원가']]

# PCA를 활용하여 지수를 만드는 것이기에, n_components 1로 고정.

pca= PCA(n_components = 1)
PC=pca.fit_transform(x)
result1 = pd.DataFrame(PC, columns=['PC1'])
result1['id']=id # pca결과에 id 데이터 추가.

# PCA PC1값에 minmax scaling을 해주었다. 
# 이유: 다중회귀분석 
min_max_scaler = MinMaxScaler(feature_range=(5,100))
x=result1['PC1']
x=x.to_numpy(dtype=float)
x=x.reshape(-1,1)
x1= min_max_scaler.fit_transform(x)
result1 = pd.DataFrame(x1, columns=['PC1'])
result1['id']=id # pca결과에 id 데이터 추가.
life_rate = result1

### 안전지수

- 가로등, 파출소, CCTV 대수

In [ ]:
# NA 값은 0개이기에, 0으로 채워준다.
safety['가로등']=safety['가로등'].fillna(0)
safety['파출소']=safety['파출소'].fillna(0)
safety['카메라대수']=safety['카메라대수'].fillna(0)

In [ ]:
# PCA 후, plot_grid_map을 하기 위해 ID가 필요하므로 미리 추출한다.
id=safety['id']
x=safety[['가로등','파출소','카메라대수']]

# PCA를 활용하여 지수를 만드는 것이기에, n_components 1로 고정.
pca= PCA(n_components = 1)
PC=pca.fit_transform(x)
result2 = pd.DataFrame(PC, columns=['PC1'])
result2['id']=id

# PCA PC1값에 minmax scaling을 해주었다. 
# 이유: 다중회귀분석 
x=result2['PC1']
x=x.to_numpy(dtype=float)
x=x.reshape(-1,1)
x1= min_max_scaler.fit_transform(x)
result2 = pd.DataFrame(x1, columns=['PC1'])
result2['id']=id 
safety_rate = result2

### 교통지수

- BRT, 광역, 지선, 간선, 마을버스 기준 그리드별 배차 횟수, 그리드별 중앙 좌표와 가까운 역, 터미널과의 거리

In [ ]:
transport['BRT']=transport['BRT'].fillna(0) # NA값 0으로 대체
transport['광역버스']=transport['광역버스'].fillna(0)
transport['지선버스']=transport['지선버스'].fillna(0)
transport['간선버스']=transport['간선버스'].fillna(0)
transport['마을버스']=transport['마을버스'].fillna(0)

In [ ]:
# PCA 후, plot_grid_map을 하기 위해 ID가 필요하므로 미리 추출한다.
id=transport['id']
x=transport[['BRT','광역버스','지선버스','간선버스','마을버스','역 터미널']]

# PCA를 활용하여 지수를 만드는 것이기에, n_components 1로 고정.
pca= PCA(n_components = 1)
PC=pca.fit_transform(x)
result3 = pd.DataFrame(PC, columns=['PC1'])
result3['id']=id

# PCA PC1값에 minmax scaling을 해주었다. 
# 이유: 다중회귀분석 
min_max_scaler = MinMaxScaler(feature_range=(5,100))
x=result3['PC1']
x=x.to_numpy(dtype=float)
x=x.reshape(-1,1)
x1= min_max_scaler.fit_transform(x)
result3 = pd.DataFrame(x1, columns=['PC1'])
result3['id']=id 
transport_rate = result3

### 생활 편의 지수

In [ ]:
# PCA 결과를 바탕으로 geo(격자) 데이터와 합쳐준다.
pca = pd.merge(life_rate,geo,on='id',how='outer')
pca = gpd.GeoDataFrame(pca)

plot_grid_map(pca, col = 'PC1', title = '생활 편의 지수', k=12,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Purples', percen = False)

### 안전 지수

In [ ]:
# PCA 결과를 바탕으로 geo(격자) 데이터와 합쳐준다.
pca = pd.merge(safety_rate,geo,on='id',how='outer')
pca = gpd.GeoDataFrame(pca)

plot_grid_map(pca, col = 'PC1', title = '안전 지수', k=12,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Greens', percen = False)

### 교통 지수

In [ ]:
# PCA 결과를 바탕으로 geo(격자) 데이터와 합쳐준다.
pca = pd.merge(transport_rate,geo,on='id',how='outer')
pca = gpd.GeoDataFrame(pca)

plot_grid_map(pca, col = 'PC1', title = '교통 지수', k=12,
              mode = 'cont_classify',c_mode = 'FisherJenks', colors = 'Blues', percen = False)

## 2) LDA

- NLP 토픽 모델링의 대표적인 알고리즘

- 문서의 집합들을 바탕으로 하며, 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 문서 내에 어떤 토픽들이 존재하는지 알아내는 알고리즘

### Modeling

In [ ]:
import gensim
from gensim import corpora
import pyLDAvis.gensim

In [ ]:
def ldamodel(df, n=4):
    # 토큰 추출
    token = df['token']
    dictionary = corpora.Dictionary(token)
    corpus = [dictionary.doc2bow(text) for text in token]
    
    #LDA 모델 훈련시키기
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = n, id2word=dictionary, passes=15)
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
    return vis

### 2017년 4월 1일 ~ 6월 30일

- 대선 주자들의 세종시 관련 공약들로 인해 세종시 주택시장에 대한 관심이 증폭되던 시기

- 세종시 주택 매매량이 증가하고 아파트 거래가 상승 폭을 보인 시기

- 아파트 거래와 동시에 인구의 유입도 많았던 시기

In [ ]:
pyLDAvis.display(ldamodel(df_2))

### 2017년 7월 1일 ~ 10월 31일

- 문재인 정권의 8.2 부동산 정책이 시행되었던 시기  
- 세종시 주택 매매 거래량이 감소하고 특히 아파트 거래가 하락 폭을 보인 시기  
- 인구의 유입도 점차 줄어들던 시기  

In [ ]:
pyLDAvis.display(ldamodel(df_3, n=3))

### 2019년 10월 1일 ~ 12월 31일
- 전국의 주택 시장이 상승장의 모습을 띄었던 시기  
- 세종시 주택 거래 중 매매거래가 폭등하고, 전월세 거래 또한 상승세의 시기  
- 폭발적으로 늘어난 주택 거래량과는 달리 순유입자수 증가율은 오히려 감세를 보였다는 점이 특징

In [ ]:
pyLDAvis.display(ldamodel(df_8))

### 2020년 5월 1일 ~ 7월 31일
- 전국의 주택 시장이 상승장의 모습을 띄었던 시기  
- 세종시 주택 거래 중 매매거래가 폭등하고, 전월세 거래 또한 상승세의 시기
- 폭발적으로 늘어난 주택 거래량과는 달리 순유입자수 증가율은 오히려 감세를 보였다는 점이 특징

In [ ]:
pyLDAvis.display(ldamodel(df_10, n=3))

## 3) Multiple Regression Analysis

- 두 개 이상의 연속형 독립변수가 연속형 종속변수에는 미치는 영향을 검증하는 분석 방법

- 독립변수 분석: 종속변수와 관련이 있는 주요 독립변수 파악

### 격자 Data  전처리

In [ ]:
# 격자데이터 읍면동별로 분류
df = gpd.sjoin(geo, data32, op='intersects', how='left')
dff = df.groupby('id').count().reset_index()
fine = dff[dff['left'] == 1][['id','left']]#하나씩으로 정상적으로 나눠진 데이터
fine = pd.merge(fine, df, on='id')[['id','ADM_DR_NM','left_x']]
dff = dff[dff['left']>1][['id','left']] #두개 이상이 할당된 데이터
df2 = pd.merge(dff, df, on='id')[['id','ADM_DR_NM','left_x']]

# 조치원 부분만 처리
eup = df2[df2['ADM_DR_NM'].str[3] == '읍']
a = eup['id'].unique().tolist()
eup = df2[df2['id'].isin(a)]
df2 = df2.drop(eup.index)
eup = eup[eup['ADM_DR_NM'].str[3] == '읍']


# 중복되는 읍면동 drop
dropdu = df2[['id','left_x']].drop_duplicates().index
df2 = df2.loc[dropdu,:]
done = pd.concat([fine, df2, eup])
emdgeo = pd.merge(done,geo, on='id')

In [ ]:
# 읍면동 지역으로 나누어서 그리드 확인
eup = emdgeo[emdgeo['ADM_DR_NM'].str[3] == '읍']
myun = emdgeo[emdgeo['ADM_DR_NM'].str[2] == '면']
dong = emdgeo[emdgeo['ADM_DR_NM'].str[2] == '동']

print('읍: ', len(eup))
print('면: ', len(myun))
print('동: ', len(dong))

In [ ]:
# 지역구분 column 추가
eup['지역 구분'] = '읍'
myun['지역 구분'] = '면'
dong['지역 구분'] = '동'
df = pd.concat([eup, myun, dong])

### 지수(생활 편의, 안전, 교통), 공시지가, 인구 데이터 Column으로 추가

In [ ]:
life_rate.columns = ['life','id']
safety_rate.columns = ['safety','id']
transport_rate.columns = ['transport','id']
gs = gongsi2[['id',2020,'변동률']]
gs.columns = ['id','공시지가','공시지가 변동률']
pop2020 = pop2020[['id','총합']].fillna(0).replace(0,1)

In [ ]:
popin = popin[['index',0]].fillna(0)
popin.columns = ['ADM_DR_NM','순전입자수']
# 순전입자수가 너무 많기 때문에 scaling 진행
min_max_scaler = MinMaxScaler(feature_range=(5,100))
x=popin['순전입자수']
x=x.to_numpy(dtype=float)
x=x.reshape(-1,1)
x1= min_max_scaler.fit_transform(x)
popin['순전입자수'] = x1

In [ ]:
age = age.replace(0,1)

In [ ]:
from functools import reduce
dfs = [df,life_rate,safety_rate,transport_rate, gs, age, pop2020]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), dfs, )
df_merge = pd.merge(df_merge, popin, on='ADM_DR_NM')
emd = df_merge.drop(['left_x','left','top','bottom','SIG_CD','SIG_KOR_NM'],axis=1)

In [ ]:
eup = emd[emd['지역 구분'] == '읍']
myun = emd[emd['지역 구분'] == '면']
dong = emd[emd['지역 구분'] == '동']

### 다중선형회귀( Level-log Model) 함수 정의

- x가 1%만큼 증가할 때 y가 $\frac{\beta}{100}$% 만큼 변하는 것을 의미하는 분석 방법 ➡︎ 독립변수를 단위화  
  
- 독립변수가 단위에 영향을 받지 않도록 행복지수, 안전지수, 교통지수에 대해서 Level-log Model을 사용

In [ ]:
from statsmodels.formula.api import ols

def log(df):
    # 공시지가 na 값 drop 후, na 아닌 데이터만 추출
    df=df.reset_index()
    df1=df['공시지가'].dropna()
    index=df1.index
    df=df.loc[index]
    df = df.fillna(0)

    # 로그 씌워주기
    df['log생활지수']=np.log(df['life'])
    df['log안전지수']=np.log(df['safety'])
    df['log교통지수']=np.log(df['transport'])
    df['log건축연한']=np.log(df['사용연도'])
    df['건축연한']=df['사용연도']
    df['log순전입자수']=np.log(df['순전입자수'])

    return df

def ourols(df, log=False):
    # 회귀 돌려주기
    res = ols('공시지가 ~ log생활지수+log안전지수+log교통지수+순전입자수', data=df).fit()
        
    return res

### 읍 (log-level)

In [ ]:
res1 = ourols(log(eup))
res1.summary()

### 면 (log-level)

In [ ]:
res2 = ourols(log(myun))
res2.summary()

### 동 (log-level)

In [ ]:
res3 = ourols(log(dong))
res3.summary()

## 4) Oaxaca-Blinder Decomposition

- 선형회귀계수를 이용해 격차를 세 가지 부분으로 분해해 격차의 발생요인을 설명하는 분석 방법

- 가격 격차가 어디서 기인하는지 분석하기 위해 Oaxaca-Blinder 분해법 적용

### Preprocessing

In [ ]:
# 읍 X와 y값 지정.
X_eup=log(eup)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_eup=log(eup)['공시지가']

# 면 X와 y값 지정.
X_myun=log(myun)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_myun=log(myun)['공시지가']

# 동 X와 y값 지정.
X_dong=log(dong)[['log생활지수','log안전지수','log교통지수','순전입자수']]
y_dong=log(dong)['공시지가']

In [ ]:
# 읍, 면 vs 동 Beta(coefficient)계수 벡터화
b_eup = np.asarray(res1.params[1:])
b_myun = np.asarray(res2.params[1:])
b_dong = np.asarray(res3.params[1:])

In [ ]:
# 읍, 면 vs 동 독립변수 mean 벡터화
x_eup = np.asarray(X_eup.mean())
x_myun = np.asarray(X_myun.mean())
x_dong = np.asarray(X_dong.mean())

In [ ]:
print('읍 공시지가 평균: ',y_eup.mean())
print('면 공시지가 평균: ',y_myun.mean())
print('동 공시지가 평균: ',y_dong.mean())

### Modeling

- Endowments: 설명 변수 차이로 인해 발생하는 격차

- Coefficients: 각 그룹의 회귀 계수 차이로 인해 발생하는 격차

- Interaction: 모델로 설명할 수 없는 부분 (잔차)

In [ ]:
def oaxaca(x1, x2, b1, b2):
    
    X=x1-x2
    B=b1-b2
    
    endowments=[]
    coefficients=[]
    interaction=[]
    
    for i in range(4):
        endowments.append(X[i]*b2[i])
        coefficients.append(x2[i]*B[i])
        interaction.append(X[i]*B[i])
    df = DataFrame()
    df['endowments'] = endowments
    df['coefficents'] = coefficients
    df['interaction'] = interaction
    df['variable'] = ['loglife','logsafety','logtransport','순전입자수']
    df = df.set_index('variable')
    df2 = DataFrame(df.transpose())
    df2['sum'] = df.sum()
    return df2

In [ ]:
oaxaca(x_dong, x_myun, b_dong, b_myun)

In [ ]:
oaxaca(x_dong, x_eup, b_dong, b_eup)